# Analysis of 2nd torus model optimization to match FC region

In [1]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Range1d, Span
from bokeh.palettes import Dark2_5 as palette
from bokeh.layouts import gridplot
from bokeh.charts import Scatter

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
import logging
logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.DEBUG)

In [4]:
import os
import sys

import numpy as np

In [5]:
iq_goal_fname = 'data/lysozyme_00001.iq'
assert os.path.exists(iq_goal_fname)
iq_goal = np.loadtxt(iq_goal_fname)

iq_model_fname = 'lys_ellipsoid/lys_ellipsoid_00001.iq'
assert os.path.exists(iq_model_fname)
iq_model = np.loadtxt(iq_model_fname)

In [6]:
p1 = figure(width=400, height=400, x_axis_label='q (1/A)', y_axis_label='I(q)', x_axis_type='log', y_axis_type='log')
p2 = figure(width=400, height=400, x_axis_label='q (1/A)', y_axis_label='I(q)')

p1.line(iq_goal[:, 0], iq_goal[:, 1], color=palette[0], line_width=2, line_dash=[4, 0], legend='experiment')
p1.line(iq_model[:, 0], iq_model[:, 1], color=palette[1], line_width=2, line_dash=[4, 4], legend='22.5x15x15 model')
p2.line(iq_goal[:, 0], iq_goal[:, 1], color=palette[0], line_width=2, line_dash=[4, 0], legend='experiment')
p2.line(iq_model[:, 0], iq_model[:, 1], color=palette[1], line_width=2, line_dash=[4, 4], legend='22.5x15x15 model')

x_range = (iq_goal[1, 0], iq_goal[-1, 0])
y_range = (min(iq_goal[:, 1]) * 0.5, iq_goal[1, 1] * 1.5)
logging.debug('x range: {}'.format(x_range))
logging.debug('y range: {}'.format(y_range))
p1.x_range = Range1d(*x_range)
p1.y_range = Range1d(*y_range)
p2.x_range = p1.x_range
p2.y_range = p1.y_range

plots = gridplot([[p1, p2]])
show(plots)

DEBUG: x range: (0.0050000000000000001, 0.29999999999999999)
DEBUG: y range: (0.0038509999999999998, 1.4977845000000001)


In [7]:
import sasmol.sasmol as sasmol
atom = sasmol.SasMol(0)
atom.read_pdb('data/lysozyme_centered.pdb')
atom_rg = atom.calcrg(0)

model = sasmol.SasMol(0)
model.read_pdb('lys_ellipsoid/lys_ellipsoid_z.pdb')
model_rg = model.calcrg(0)

logging.debug('atomistic Rg: {}'.format(atom_rg))
logging.debug('model Rg: {}'.format(model_rg))

reading filename:  data/lysozyme_centered.pdb
num_atoms =  1960
>>> found  1  model(s) or frame(s)
finished reading frame =  1
reading filename:  lys_ellipsoid/lys_ellipsoid_z.pdb
num_atoms =  2000
>>> found  1  model(s) or frame(s)
finished reading frame =  1


DEBUG: atomistic Rg: 13.9946706991
DEBUG: model Rg: 13.8646412507


In [9]:
import pandas as pd
run_dir = 'lys_ellipsoid_scan1'

log_fname = os.path.join(run_dir, 'sx13to17_sy13to17_sz20.5to24.5.log')
log = pd.read_csv(log_fname, delimiter='\t', index_col=[0], escapechar='#')
log.rename(columns=lambda x: x.strip(), inplace=True)

p_fname = os.path.join(run_dir, 'sascalc/neutron_D2Op_100/pdb_percent_error.txt')
p = pd.read_csv(p_fname, delimiter='\t', index_col=[0])
p.rename(columns=lambda x: x.strip(), inplace=True)

In [10]:
p.head()

,p,fnames
1,15.216252,lys_ellipsoid_scan1/sascalc/neutron_D2Op_100/l...
2,9.290326,lys_ellipsoid_scan1/sascalc/neutron_D2Op_100/l...
3,8.631331,lys_ellipsoid_scan1/sascalc/neutron_D2Op_100/l...
4,15.785384,lys_ellipsoid_scan1/sascalc/neutron_D2Op_100/l...
5,16.222873,lys_ellipsoid_scan1/sascalc/neutron_D2Op_100/l...


In [11]:
log.head()

,rg,sx,sy,sz
frame,,,,
1,12.320748,13.0,13.0,20.5
2,12.507873,13.0,14.0,20.5
3,12.706000,13.0,15.0,20.5
4,13.069334,13.0,16.0,20.5
5,13.311778,13.0,17.0,20.5


In [12]:
result = pd.concat([log, p], axis=1)

In [13]:
result.head()

,rg,sx,sy,sz,p,fnames
frame,,,,,,
1,12.320748,13.0,13.0,20.5,15.216252,lys_ellipsoid_scan1/sascalc/neutron_D2Op_100/l...
2,12.507873,13.0,14.0,20.5,9.290326,lys_ellipsoid_scan1/sascalc/neutron_D2Op_100/l...
3,12.706000,13.0,15.0,20.5,8.631331,lys_ellipsoid_scan1/sascalc/neutron_D2Op_100/l...
4,13.069334,13.0,16.0,20.5,15.785384,lys_ellipsoid_scan1/sascalc/neutron_D2Op_100/l...
5,13.311778,13.0,17.0,20.5,16.222873,lys_ellipsoid_scan1/sascalc/neutron_D2Op_100/l...


In [15]:
i_best = result.p.idxmin()
i_worst = result.p.idxmax()
logging.info('\nbest: {}\nworst: {}'.format(i_best, i_worst))

INFO: 
best: 101
worst: 125


In [16]:
result[result.sz == 22.5]

,rg,sx,sy,sz,p,fnames
frame,,,,,,
51,13.072000,13.0,13.0,22.5,6.394346,lys_ellipsoid_scan1/sascalc/neutron_D2Op_100/l...
52,13.169300,13.0,14.0,22.5,7.655557,lys_ellipsoid_scan1/sascalc/neutron_D2Op_100/l...
53,13.434784,13.0,15.0,22.5,12.742449,lys_ellipsoid_scan1/sascalc/neutron_D2Op_100/l...
54,13.612343,13.0,16.0,22.5,18.177884,lys_ellipsoid_scan1/sascalc/neutron_D2Op_100/l...
55,13.655938,13.0,17.0,22.5,25.892210,lys_ellipsoid_scan1/sascalc/neutron_D2Op_100/l...
56,13.336388,14.0,13.0,22.5,6.671852,lys_ellipsoid_scan1/sascalc/neutron_D2Op_100/l...
57,13.282775,14.0,14.0,22.5,12.358976,lys_ellipsoid_scan1/sascalc/neutron_D2Op_100/l...
58,13.706066,14.0,15.0,22.5,20.652723,lys_ellipsoid_scan1/sascalc/neutron_D2Op_100/l...
59,13.754028,14.0,16.0,22.5,24.882970,lys_ellipsoid_scan1/sascalc/neutron_D2Op_100/l...


In [17]:
result.columns

Index([u'rg', u'sx', u'sy', u'sz', u'p', u'fnames'], dtype='object')

In [18]:
import glob

In [19]:
goal_fname = 'data/lysozyme_00001.iq'
goal_iq = np.loadtxt(goal_fname)[:, :2]
nq, _ = goal_iq.shape

In [20]:
run_dir = 'lys_ellipsoid_scan1/sascalc/neutron_D2Op_100/'
ext = '*.iq'
file_search = os.path.join(run_dir, ext)
iq_fnames = glob.glob(file_search)
assert iq_fnames, 'ERROR: no files found using {}'.format(file_search)
iq_fnames.sort()

iq_data = np.empty((nq, len(iq_fnames)+1), order='F')
iq_data[:, 0] = goal_iq[:, 0]
for i, fname in enumerate(iq_fnames):
    this_iq_data = np.loadtxt(fname)
    iq_data[:, i+1] = this_iq_data[:, 1] * iq_goal[0, 1] / this_iq_data[0, 1]

In [31]:
p1 = Scatter(result, x='rg', y='p', color=palette[1], width=400, height=400, 
             xlabel='Rg (A)', ylabel='R-factor')
vline = Span(location=atom_rg, dimension='height', line_color=palette[0], line_width=3, )
p1.renderers.extend([vline])

p2 = figure(width=400, height=400, x_axis_label='q (1/A)', y_axis_label='I(q)') #, x_axis_type='log', y_axis_type='log')

# for i in xrange(len(iq_fnames)):
#     p2.line(iq_data[:, 0], iq_data[:, i+1], color='black', legend='scan', line_width=2)
p2.line(iq_goal[:, 0], iq_goal[:, 1], color=palette[0], line_width=2, line_dash=[6, 6], 
        legend='experiment')
p2.line(iq_data[:, 0], iq_data[:, i_best], color=palette[1], line_width=2, line_dash=[5, 1], 
        legend='best')
p2.line(iq_model[:, 0], iq_model[:, 1], color=palette[2], line_width=2, 
        legend='22.5x15x15 model')
p2.line(iq_data[:, 0], iq_data[:, i_worst], color=palette[3], line_width=2, line_dash=[4, 2, 2], 
        legend='worst')

x_range = (iq_goal[1, 0], iq_goal[-1, 0])
y_range = (min(iq_goal[:, 1]) * 0.5, iq_goal[1, 1] * 1.5)
logging.debug('x range: {}'.format(x_range))
logging.debug('y range: {}'.format(y_range))
p2.x_range = Range1d(*x_range)
p2.y_range = Range1d(*y_range)
# p2.legend.location = 'bottom_left'

plots = gridplot([[p1, p2]])
show(plots)

DEBUG: x range: (0.0050000000000000001, 0.29999999999999999)
DEBUG: y range: (0.0038509999999999998, 1.4977845000000001)
